In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# System Configs

In [ ]:
!nvidia-smi


In [ ]:
import tensorflow as tf

# Check if GPU is available
  # Check if TensorFlow detects a GPU
    print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
    print("GPU Devices:", tf.config.list_physical_devices('GPU'))
else:
    print("No GPU found.")
    gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"GPUs available: {[gpu.name for gpu in gpus]}")
  
    


In [ ]:
!pip install torch==2.5.1+cu121 torchvision==0.20.1+cu121 torchaudio==2.5.1+cu121 --index-url https://download.pytorch.org/whl/cu121


In [ ]:
import torch
print(torch.__version__)  # PyTorch version
print(torch.version.cuda)  # CUDA version used by PyTorch
print(torch.cuda.is_available())  # Check if GPU is accessible


In [ ]:
%%capture
# !pip install unsloth
# Also get the latest nightly Unsloth!

    
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
!pip install bitsandbytes triton unsloth-zoo xformers 

In [1]:
!python --version
!pip show torch | grep Version
!python -m torch.utils.collect_env


Python 3.10.14
Version: 2.4.0+cpu
/opt/conda/lib/python3.10/runpy.py:126: RuntimeWarning: 'torch.utils.collect_env' found in sys.modules after import of package 'torch.utils', but prior to execution of 'torch.utils.collect_env'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
PyTorch version: 2.4.0+cpu
Is debug build: False
CUDA used to build PyTorch: None
ROCM used to build PyTorch: N/A

OS: Ubuntu 20.04.6 LTS (x86_64)
GCC version: (Ubuntu 9.4.0-1ubuntu1~20.04.2) 9.4.0
Clang version: Could not collect
CMake version: version 3.16.3
Libc version: glibc-2.31

Python version: 3.10.14 | packaged by conda-forge | (main, Mar 20 2024, 12:45:18) [GCC 12.3.0] (64-bit runtime)
Python platform: Linux-6.6.56+-x86_64-with-glibc2.31
Is CUDA available: False
CUDA runtime version: No CUDA
CUDA_MODULE_LOADING set to: N/A
GPU models and configuration: No CUDA
Nvidia driver version: No CUDA
cuDNN version: No CUDA
HIP runtime version: N/A
MIOpen runtime version: N/A
Is XNNPACK avail

# Libraries

In [13]:

import json
import pandas as pd
import wandb
from unsloth import FastLanguageModel

# Data Imports

In [30]:
df1 = pd.read_parquet('/kaggle/input/agriculture-qa/train-00000-of-00001.parquet')
df2 = pd.read_json("/kaggle/input/agriculture-qa/agricult_data.json")

In [31]:
df2 = df2.drop(columns=['instruction'])
df2.columns = df1.columns
df = pd.concat([df1, df2], ignore_index=True)
df

,question,answers
0,why is crop rotation important in farming?,This helps to prevent soil erosion and depleti...
1,What farming practice helps prevent soil erosion?,Crop Rotation
2,what is crop rotation,Crop rotation is the practice of growing a ser...
3,what are the different methods of irrigation?,"surface irrigation, drip irrigation, and sprin..."
4,why is soil health vital?,Soil health is critical to crop growth and pro...
...,...,...
28526,Addressing nitrogen deficiency in a newly esta...,Intersperse nitrogen-fixing plants among fruit...
28527,Preventing erosion on riverbanks used for cult...,Plant vetiver grass along the banks for its de...
28528,Reviving an abandoned vineyard with minimal in...,Introduce ground cover crops to improve soil l...
28529,Managing waterlogged fields after heavy rains.,Construct raised beds or channels to improve d...


# Data Preprocess

In [32]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True


In [ ]:
# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "unsloth/Meta-Llama-3.1-8B",
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
#     # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
# )
# model.save_pretrained("lora_model") # Local saving
# tokenizer.save_pretrained("lora_model")

# Model

In [33]:

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.11.8: Fast Llama patching. Transformers = 4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [64]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.11.8 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
# alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.


# ### Input:
# {}

# ### Response:
# {}"""

# EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
# def formatting_prompts_func(examples):
    
#     inputs       = examples["question"]
#     outputs      = examples["answers"]
#     texts = []
#     for input, output in zip(inputs, outputs):
#         # Must add EOS_TOKEN, otherwise your generation will go on forever!
#         text = alpaca_prompt.format(input, output) + EOS_TOKEN
#         texts.append(text)
#     return { "text" : texts, }



In [35]:
from datasets import Dataset

# Prepare conversations for formatting
conversations = [
    [{"role": "user", "content": row["question"]}, {"role": "assistant", "content": row["answers"]}]
    for _, row in df.iterrows()
]

# Create a Hugging Face Dataset
hf_dataset = Dataset.from_dict({"conversations": conversations})
hf_dataset

Dataset({
    features: ['conversations'],
    num_rows: 28531
})

In [58]:
hf_dataset[:2]

{'conversations': [[{'content': 'why is crop rotation important in farming?',
    'role': 'user'},
   {'content': 'This helps to prevent soil erosion and depletion, and can also help to control pests and diseases',
    'role': 'assistant'}],
  [{'content': 'What farming practice helps prevent soil erosion?',
    'role': 'user'},
   {'content': 'Crop Rotation', 'role': 'assistant'}]]}

In [59]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }


# Apply the formatting function
formatted_dataset = hf_dataset.map(formatting_prompts_func, batched=True)


Map:   0%|          | 0/28531 [00:00<?, ? examples/s]

In [61]:
hf_dataset[2]

{'conversations': [{'content': 'what is crop rotation', 'role': 'user'},
  {'content': 'Crop rotation is the practice of growing a series of different crops in the same area over several seasons',
   'role': 'assistant'}]}

In [40]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
token = user_secrets.get_secret("HUGGINGFACE_TOKEN")


In [51]:
hf_dataset.push_to_hub("ShuklaShreyansh/Agriculture-QA",token=token)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/29 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/datasets/ShuklaShreyansh/Agriculture-QA/commit/9c5925307b0bd653f12b80d28c5e9683612be889', commit_message='Upload dataset', commit_description='', oid='9c5925307b0bd653f12b80d28c5e9683612be889', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/ShuklaShreyansh/Agriculture-QA', endpoint='https://huggingface.co', repo_type='dataset', repo_id='ShuklaShreyansh/Agriculture-QA'), pr_revision=None, pr_num=None)

In [ ]:
!wandb login
# !wandb login --relogin

# Training

In [66]:

import torch
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = hf_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
#         max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "wandb", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/28531 [00:00<?, ? examples/s]

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])


In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

In [ ]:
trainer_stats = trainer.train()

# Save/Load

In [15]:
model.save_pretrained("model") # Local saving
model.save_pretrained_merged("model")
tokenizer.save_pretrained("model")

Unsloth: You're not saving a tokenizer as well?
You can do it separately via `tokenizer.save_pretrained(...)`
Unsloth: You have 2 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 2.2G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 18.83 out of 31.35 RAM for saving.


100%|██████████| 28/28 [00:00<00:00, 255.36it/s]



Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving model/pytorch_model-00001-of-00002.bin...
Unsloth: Saving model/pytorch_model-00002-of-00002.bin...
Done.


('model/tokenizer_config.json',
 'model/special_tokens_map.json',
 'model/tokenizer.json')

In [12]:
# Now if you want to load the LoRA adapters we just saved for inference, set False to True:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True


from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/kaggle/working/fine-tuned Llama-3.2-3B-Instruct", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)


==((====))==  Unsloth 2024.11.7: Fast Llama patching. Transformers = 4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Save to HuggingFace_hub

In [38]:
# Push model to the hub
token = os.getenv("HUGGINGFACE_TOKEN")
model.push_to_hub("ShuklaShreyansh/Agro-QA", token=token)
model.push_to_hub_merged("ShuklaShreyansh/Agro-QA", token=token)
tokenizer.push_to_hub("ShuklaShreyansh/Agro-QA", token=token)

  0%|          | 0/1 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Saved model to https://huggingface.co/ShuklaShreyansh/Agro-QA
Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 19.68 out of 31.35 RAM for saving.


100%|██████████| 28/28 [00:04<00:00,  6.70it/s]



Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving /tmp/Agro-QA/pytorch_model-00001-of-00002.bin...
Unsloth: Saving /tmp/Agro-QA/pytorch_model-00002-of-00002.bin...


No files have been modified since last commit. Skipping to prevent empty commit.


Done.
Saved merged model to https://huggingface.co/ShuklaShreyansh/Agro-QA


No files have been modified since last commit. Skipping to prevent empty commit.


# Inference

In [18]:
# from unsloth.chat_templates import get_chat_template

# tokenizer = get_chat_template(
#     tokenizer,
#     chat_template = "llama-3.1",
# )
# FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# messages = [
#     {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
# ]
# inputs = tokenizer.apply_chat_template(
#     messages,
#     tokenize = True,
#     add_generation_prompt = True, # Must add for generation
#     return_tensors = "pt",
# ).to("cuda")

# outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
#                          temperature = 1.5, min_p = 0.1)
# tokenizer.batch_decode(outputs)

In [18]:
# FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# messages = [
#     {"role": "user", "content": "What is Agriculture?"},
# ]
# inputs = tokenizer.apply_chat_template(
#     messages,
#     tokenize = True,
#     add_generation_prompt = True, # Must add for generation
#     return_tensors = "pt",
# ).to("cuda")

# from transformers import TextStreamer
# text_streamer = TextStreamer(tokenizer, skip_prompt = True)
# _ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
#                    use_cache = True, temperature = 1.5, min_p = 0.1)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Agriculture is the practice of cultivating plants and animals on a farm or other landholdings to produce food, feed, and other products. It involves various activities such as planting, harvesting, and processing crops, as well as raising livestock, managing soil quality, and controlling pests and diseases.

The primary goal of agriculture is to produce a steady supply of food and other agricultural products to feed and sustain human populations. However, modern agriculture also aims to provide other benefits, including:

1. Environmental sustainability: This includes practices like conservation tillage, crop rotation, and integrated pest management (IPM) to reduce soil erosion, promote biodiversity,


# Inference from HuggingFace

In [8]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

token = user_secrets.get_secret("HUGGINGFACE_TOKEN")
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True
model_name = "ShuklaShreyansh/Agro-QA"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = token, # use one if using gated models like meta-llama/Llama-2-7b-hf
)



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.11.8: Fast Llama patching. Transformers = 4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [11]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "What is Agriculture?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Agriculture is the practice of cultivating the land and producing crops and livestock to feed a population. It involves the management of agricultural resources, such as land, water, soil, and living organisms, to produce food, fiber, and other agricultural products. Agriculture is a vital component of the global economy, providing a significant portion of the world's food, and its significance cannot be overstated.

There are several key aspects of agriculture:

1. **Crop production**: Planting, growing, and harvesting crops such as grains, fruits, and vegetables.
2. **Livestock production**: Raising animals, such as cattle, pigs,
